# Deeper Contexts: LangGraph + Content-Centric Knowledge Graphs

## Introduction

Knowledge graphs are a popular for use in RAG (Retrieval Augmented Generation). These techniques often involve extracting the structured graph -- entities and relationships -- using an LLM, and then retrieving information from the knowledge graph and feeding it to the LLM as context for answering questions.

We've introduced [content-centric knowledge graphs](...) as an improvement to knowledge graphs focused on RAG. Rather than extracting detailed structure, a content-centric knowledge graph adds links between vector chunks, providing the ability to navigate links for deeper information without incurring an expensive and hands-on indexing process. We implemented this concept in LangChain as [GraphVectorStore](...).

One huge benefit of the knowledge graph is allowing the LLM to explore information in much the same way we would. While traversals allow exploring part of the graph to answer a specific question, another important capability is the ability to ask follow-up questions. [Knowledge Graph Prompting for Multi-Document Question Answering](https://arxiv.org/abs/2308.11730) demonstrated the benefits of being able to limit follow-up questions using the graph structure.

As an example -- if you read a section of a book and don't understand some of the terms, is it better to consult a dictionary or that books glossary? If you go to the dictionary, you *may* find the answer and be able to relate it to what you read... but if you look in the glossary, you'll get the definition that is used within the book, making it much easier to understand the context of what you have already read.

In this post we'll use LangGraph to build an agent and provide it with tools to perform an initial retrieval (using [MMR graph traversal](...)) as well as a tool for asking a follow-up question "in the neighborhood" of already retrieved documents.

## Colab Environment Setup

The following block will configure the environment from the Colab Secrets.
To run it, you should have the following Colab Secrets defined and accessible to this notebook:

- `OPENAI_API_KEY`: The OpenAI key.
- `ASTRA_DB_DATABASE_ID`: The Astra DB database ID.
- `ASTRA_DB_APPLICATION_TOKEN`: The Astra DB Application token.
- `LANGCHAIN_API_KEY`: Optional. If defined, will enable LangSmith tracing.
- `ASTRA_DB_KEYSPACE`: Optional. If defined, will specif the Astra DB keyspace. If not defined, will use thee default.

In [ ]:
# Install modules.
%pip install -U -r requirements.txt

In [ ]:
# Override knowledge store from my fork.
# Won't be needed once this is available.
%pip install --force-reinstall git+https://github.com/bjchambers/langchain.git@cassandra-graph-vectorstore-kwargs#subdirectory=libs/community
%pip install --force-reinstall git+https://github.com/datastax/ragstack-ai.git@timeout#subdirectory=libs/knowledge-store

In [ ]:
# Configure import paths.
import sys
sys.path.append("../../")

# Initialize environment variables.
from utils import initialize_environment
initialize_environment()

In [ ]:
#@ GraphVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.graph_vectorstores.cassandra import CassandraGraphVectorStore
import cassio

cassio.init(auto=True)
store = CassandraGraphVectorStore(
    embedding = OpenAIEmbeddings(),
    node_table="neighborhood_nodes",
    insert_timeout = 1000.0,
)


## Load Initial Data

This only needs to be done once to populate the datastore. Since the focus of this notebook is on using the knowledge graph within an agent, I'll just do all of the loading in a single code block, below. Refer to the comments if interseted, and/or the documentation for populating a `GraphVectorStore`.

In [ ]:
#@ Load Data Into the Graph VectorStore
from datasets.wikimultihop.load import load_2wikimultihop
load_2wikimultihop(store)

## The Tools

Each of the tools will be set up to return a Pydantic model representing the retrieved document chunks. Each chunk will include the `chunk_id`, the `document_id` (the document the chunk is from), and the `page_content`.

The initial retrieval tool will take the question and return the representative chunks. The neighborhood retrieval tool will take a question and a list of `document_ids`. It will retrieve nodes matching the question from the graph starting with those adjacent to `document_ids`.

This combination allows the agent to formulate one or more initial questions based on the user's request, and then ask follow-up questions as needed to better understand the information retrieved.

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool
from typing import List, Sequence, Any

class InitialRetrieveInput(BaseModel):
    question: str = Field(description="Question to retrieve content for. Should be a simple question describing the starting point for retrieval likely to have content.")

class FollowupRetrieveInput(BaseModel):
    question: str = Field(descri    ption="Question to retrieve follow-up content for.")
    neighborhood: List[str] = Field(default = None, description="Content IDs that should be used to limit the follow-up retrieval. Only information linked from the neighborhood will be retrieved.")

# TODO: We could also have a follow-up that just limits to the *source* IDs.
# TODO: We could combine these two tools with an `Optional[List[str]]` neighborhood.

class RetrievedContent(BaseModel):
    content_id: str = Field(description="ID of this chunk of content")
    content: str = Field(description = "Content of this chunk")

class RetrievedContext(BaseModel):
    question: str = Field(description = "question this context was answering")
    contents: List[RetrievedContent] = Field(description="retrieved content")

def _retrieve(question: str,
              *,
              neighborhood: Sequence[str] = (),
              **kwargs: Any) -> RetrievedContext:
    documents = store.search(
        query = question,
        search_type = "mmr_traversal",
        initial_roots = neighborhood,
        fetch_k = 0 if neighborhood else 100,
        **kwargs,
    )
    return RetrievedContext(
        question = question,
        contents = [
            RetrievedContent(content_id=doc.id, content=doc.page_content)
            for doc in documents
        ]
    )

def initial_retrieve(question: str) -> RetrievedContext:
    return _retrieve(question, depth=0)

def followup_retrieve(question: str, neighborhood: List[str]) -> RetrievedContext:
    return _retrieve(question, neighborhood=neighborhood)

initial_retrieve_tool = StructuredTool.from_function(
    func=initial_retrieve,
    name="InitialRetrieve",
    description="Retrieve context answering a specific question. Use when there isn't a known neighborhood to search in.",
    args_schema=InitialRetrieveInput,
    return_direct=False,
    # coroutine= ... <- you can specify an async method if desired as well
)

followup_retrieve_tool = StructuredTool.from_function(
    func=followup_retrieve,
    name="FollowupRetrieve",
    description="Retrieve context answering a follow-up question using information near existing content. Only use when there are likely links from existing content to the desired follow-up content.",
    args_schema=FollowupRetrieveInput,
    return_direct=False,
)

## The Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [
    initial_retrieve_tool,
    followup_retrieve_tool,
]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the agent executor (the actual runnable)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example 1: No follow-up questions needed
We use a question from the 2wikimultihop test set -- specifically "Where was the place of death of the directory of the film Ladies Courageous?".
This normally requires multiple hops -- first ask who directed the film "Ladies Courageous", then ask where John Rawlins died.

In [ ]:
store.store._session.default_timeout = 60.0
agent_executor.invoke({"input": "Where was the place of death of the director of the film Ladies Courageous?"})

Looking at the output, we see that the agent retrieved results for an initial question -- "Who was the director of the film Ladies Courageous".
The results included the Wikipedia page about Ladies Courageous, but also included the page about John Rawlins.
This is because it was linked to by the first page, and the MMR graph retrieval determined the results were relevant (John Rawlins was the directory).

This is interesting, because it didn't require a second hop at all, thanks to the graph edges!

## Example 2: Multiple Hops


In [ ]:
agent_executor.invoke({"input": "Which film has the director who died earlier, The Boy Turns Man or A Strange Adventure?"})

## Conclusion
To be written.